In [2]:
import torch
from torch import nn
import sklearn
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
import numpy as np
from skorch import NeuralNetClassifier
import itertools


/home/jaren/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:

# Extract Dataset
input_file = "Data/features_30_sec.csv"
with open(input_file) as f:
    f.readline()
    data = np.loadtxt(f, delimiter=',', dtype=np.object_)

labels = [
    'blues', 'classical', 'country', 'disco', 'hiphop', 
    'jazz', 'metal', 'pop', 'reggae', 'rock'
]

X = data[:, 1:-1].astype(np.float32)

min_max_scaler = preprocessing.MinMaxScaler((-1, 1))
X = min_max_scaler.fit_transform(X)

y = [labels.index(l) for l in data[:, -1]]


train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=42)

train_X = torch.tensor(train_X, dtype=torch.float32)
test_X  = torch.tensor(test_X, dtype=torch.float32)
train_y = torch.tensor(train_y, dtype=torch.long)
test_y  = torch.tensor(test_y, dtype=torch.long)


In [10]:
torch.manual_seed(123456789)

# class Network(nn.Module):
#     def __init__(self, in_features: int, out_features: int):
#         super().__init__()
#         self.l1 = nn.Linear(in_features, 500)
#         self.l2= nn.Linear(500, 1000)        
#         self.dropout = nn.Dropout(0.5)
#         self.l3 = nn.Linear(1000, 500)
#         self.l4 = nn.Linear(500, out_features)

#     def forward(self, x):
#         output = torch.sigmoid(self.l1(x))
#         output = torch.sigmoid(self.l2(output))
#         output = self.dropout(output)
#         output = torch.sigmoid(self.l3(output))
#         output = self.l4(output)
#         output = output
#         return output

class Network(nn.Module):
    def __init__(self, in_features: int, out_features: int, dropout: float = 0.5):
        super().__init__()
        # self.l1 = nn.Linear(in_features, 128)
        # self.dropout = nn.Dropout(0.5)
        # self.l2 = nn.Linear(128, out_features)

        self.dropout = nn.Dropout(dropout)

        self.l1 = nn.Linear(in_features, 256)
        self.batch_norm1 = nn.BatchNorm1d(256)
        self.l2 = nn.Linear(256, 64)
        self.l3 = nn.Linear(64, out_features)


    def forward(self, x):
        output = torch.relu(self.l1(x))
        output = self.dropout(output)
        output = self.batch_norm1(output)
        output = torch.relu(self.l2(output))
        output = self.dropout(output)
        output = torch.log_softmax(self.l3(output), dim=1)



        return output




class Thing:
    def __init__(
        self, 
        module: nn.Module, 
        lr: float = 0.001, 
        weight_decay: float = .1, 
        epochs: int=100
    ):
        self.module = module
        self.lr = lr
        self.weight_decay = weight_decay
        self.epochs = epochs

    def fit(self, X, y, log: bool = True):
        criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(self.module.parameters(), lr=self.lr, weight_decay=self.weight_decay)

        transformed_labels = torch.zeros(X.shape[0], len(labels))

        for i, label in enumerate(y):
            transformed_labels[i][label.item()] = 1

        for epoch in range(self.epochs):

            y_hat = self.module(X)
            loss = criterion(y_hat, transformed_labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if log and (epoch+1) % 25 == 0:
                print(f'Epoch [{epoch+1}/{self.epochs}], Loss: {loss.item():.4f}, Accuracy: {accuracy_score(y, torch.max(y_hat, 1)[1]):.4f}')

    def predict(self, X):
        return torch.max(self.module(X), 1)


def run():
    thing = Thing(Network(58, 10), .001, .001, 500)
    thing.fit(train_X, train_y)
    score, predicted = thing.predict(test_X)
    accuracy = accuracy_score(test_y, predicted)

    print(accuracy)



def grid_search():
    for lr, wd in itertools.product((.01, .001, .0001), (.1, .01, .001, .0001, 0)):
        thing = Thing(Network(58, 10), lr, wd , 500)
        thing.fit(train_X, train_y, False)
        score, predicted = thing.predict(test_X)
        accuracy = accuracy_score(test_y, predicted)

        print(lr, wd, accuracy)


run()
# grid_search()



Epoch [25/500], Loss: 1.5945, Accuracy: 0.4462
Epoch [50/500], Loss: 1.1748, Accuracy: 0.6075
Epoch [75/500], Loss: 0.8694, Accuracy: 0.7137
Epoch [100/500], Loss: 0.6729, Accuracy: 0.7700
Epoch [125/500], Loss: 0.5237, Accuracy: 0.8287
Epoch [150/500], Loss: 0.4277, Accuracy: 0.8675
Epoch [175/500], Loss: 0.3615, Accuracy: 0.8900
Epoch [200/500], Loss: 0.3013, Accuracy: 0.9038
Epoch [225/500], Loss: 0.2601, Accuracy: 0.9187
Epoch [250/500], Loss: 0.2414, Accuracy: 0.9263
Epoch [275/500], Loss: 0.2215, Accuracy: 0.9263
Epoch [300/500], Loss: 0.2005, Accuracy: 0.9400
Epoch [325/500], Loss: 0.1793, Accuracy: 0.9487
Epoch [350/500], Loss: 0.1687, Accuracy: 0.9525
Epoch [375/500], Loss: 0.1381, Accuracy: 0.9650
Epoch [400/500], Loss: 0.1193, Accuracy: 0.9738
Epoch [425/500], Loss: 0.1520, Accuracy: 0.9525
Epoch [450/500], Loss: 0.1137, Accuracy: 0.9688
Epoch [475/500], Loss: 0.0877, Accuracy: 0.9812
Epoch [500/500], Loss: 0.1249, Accuracy: 0.9688
0.71


In [12]:
classifier = NeuralNetClassifier(
    module=Network,
    max_epochs=500,
    criterion=nn.CrossEntropyLoss,
    optimizer=torch.optim.Adam,
    lr=0.01,
    module__in_features=58,
    module__out_features=10,
    optimizer__weight_decay=0.001,
    verbose=0
)

classifier.fit(train_X, train_y)
prediction_probs = classifier.predict(test_X)

print(accuracy_score(test_y, prediction_probs))

0.65


In [6]:
from sklearn.model_selection import GridSearchCV

# ((.01, .001, .0001), (.1, .01, .001, .0001, 0)

grid = {
    'lr': [.01, .001, .0001],
    'optimizer__weight_decay': [.01, .001, .0001],
    'module__dropout': [.3, .5, .7]
}
gs = GridSearchCV(classifier, grid, refit=True, cv=5, scoring='accuracy', verbose=4)
gs.fit(train_X, train_y)

print(gs.best_score_, gs.best_params_)


Fitting 5 folds for each of 27 candidates, totalling 135 fits
[CV 1/5] END lr=0.01, module__dropout=0.3, optimizer__weight_decay=0.01;, score=0.681 total time=   5.3s
[CV 2/5] END lr=0.01, module__dropout=0.3, optimizer__weight_decay=0.01;, score=0.725 total time=   5.4s
[CV 3/5] END lr=0.01, module__dropout=0.3, optimizer__weight_decay=0.01;, score=0.700 total time=   5.0s
[CV 4/5] END lr=0.01, module__dropout=0.3, optimizer__weight_decay=0.01;, score=0.700 total time=   5.0s
[CV 5/5] END lr=0.01, module__dropout=0.3, optimizer__weight_decay=0.01;, score=0.650 total time=   7.0s
[CV 1/5] END lr=0.01, module__dropout=0.3, optimizer__weight_decay=0.001;, score=0.675 total time=   5.3s
[CV 2/5] END lr=0.01, module__dropout=0.3, optimizer__weight_decay=0.001;, score=0.713 total time=   5.5s
[CV 3/5] END lr=0.01, module__dropout=0.3, optimizer__weight_decay=0.001;, score=0.731 total time=   6.1s
[CV 4/5] END lr=0.01, module__dropout=0.3, optimizer__weight_decay=0.001;, score=0.706 total ti